In [1]:
import os
import numpy as np
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt

data_path = '/Users/jiahang/Documents/gnn/dataset'

In [3]:
path = os.path.join(data_path, 'FC_Fisher_Z_transformed.pkl')
with open(path, 'rb') as f:
    data_FC = pickle.load(f)

path = os.path.join(data_path, 'SC.pkl')
with open(path, 'rb') as f:
    data_SC = pickle.load(f)

path = os.path.join(data_path, 'T1.pkl')
with open(path, 'rb') as f:
    data_T1 = pickle.load(f)

path = os.path.join(data_path, 'demo.pkl')
with open(path, 'rb') as f:
    data_demo = pickle.load(f)

In [40]:
len(data_FC)

528

In [39]:
len(data_SC)

636

In [7]:
len(data_T1)

642

In [8]:
len(data_demo)

528

In [10]:
cnt_fc = 0
for name_fc in data_FC.keys():
    if name_fc not in data_SC.keys():
        cnt_fc += 1

In [16]:
cnt_fc # there are 14 fc which are not in sc

14

In [12]:
cnt_sc = 0
for name_sc in data_SC.keys():
    if name_sc not in data_FC.keys():
        cnt_sc += 1

In [15]:
cnt_sc # there are 122 sc which are not in fc

122

In [35]:
cnt = 0
for key in set(list(data_SC.keys()) + list(data_FC.keys())):
    if key not in data_SC.keys() or key not in data_FC.keys():
        cnt += 1
print(cnt)

136


In [22]:
data_demo['HCD0008117']

dict_keys(['Age in month', 'Sex', 'Age in year', 'race', 'ethnic_group', 'site', 'perc', 'fd'])

In [36]:
136 / 650

0.20923076923076922

In [27]:
cnt = 0
for key, val in data_demo.items():
    if 'nih_totalcogcomp_ageadjusted' in val.keys():
        cnt += 1
print(cnt)

382


In [52]:
len(np.unique(list(data_SC.keys()))) == len(data_SC.keys())

True

In [53]:
len(np.unique(list(data_FC.keys()))) == len(data_FC.keys())

True

In [54]:
whole_set = np.unique(list(data_SC.keys()) + list(data_FC.keys()))

In [55]:
# how many graphs having SC but not having FC? not considering having labels or not
cnt = 0
for key in whole_set:
    if key not in data_FC.keys() and key in data_SC.keys():
        cnt += 1
print(cnt)

122


In [56]:
# how many graphs having FC but not having SC? not considering having labels or not
cnt = 0
for key in whole_set:
    if key not in data_SC.keys() and key in data_FC.keys():
        cnt += 1
print(cnt)

14


In [61]:
# in labeled data, how many graphs having SC but not having FC
cnt = 0
for key in whole_set:
    if key not in data_FC.keys() and key in data_SC.keys() and key in data_demo.keys() and 'nih_totalcogcomp_ageadjusted' in data_demo[key].keys() :
        cnt += 1
print(cnt)

0


In [62]:
# in labeled data, how many graphs having FC but not having SC
cnt = 0
for key in whole_set:
    if key not in data_SC.keys() and key in data_FC.keys() and key in data_demo.keys() and 'nih_totalcogcomp_ageadjusted' in data_demo[key].keys() :
        cnt += 1
print(cnt)

10


In [ ]:
cnt = 0
for key, val in data_SC.items():
    if key in data_FC.keys():
        cnt += 1
print(cnt)

In [28]:
cnt = 0
for key, val in data_demo.items():
    if 'nih_totalcogcomp_ageadjusted' in val.keys() and key in data_SC.keys():
        cnt += 1
print(cnt)

372


In [29]:
cnt = 0
for key, val in data_demo.items():
    if 'nih_totalcogcomp_ageadjusted' in val.keys() and key in data_FC.keys():
        cnt += 1
print(cnt)

382


# test knn speed

In [3]:
from dgl import knn_graph as dgl_knn_graph
from torch_geometric.nn.pool import knn as pyg_knn
from torch_geometric.nn.pool import knn_graph as pyg_knn_graph

In [5]:
import sys
sys.path.append("/home/jiahang/gnn")

In [9]:
import pickle
file_path = "/home/jiahang/gnn/dataset/processed_data_regression_split.pkl"
print(f"read processed data from {file_path}")
with open(file_path, 'rb') as f:
    data = pickle.load(f)
adjs = data['adjs']
raw_Xs = data['raw_Xs']
labels = data['labels']
mu_lbls = data['mu_lbls']
std_lbls = data['std_lbls']

read processed data from /home/jiahang/gnn/dataset/processed_data_regression_split.pkl


In [22]:
%%timeit -n5 -r5
print(1)

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
9.91 µs ± 4.4 µs per loop (mean ± std. dev. of 5 runs, 5 loops each)


## dgl_knn_graph

In [23]:
adjs_dgl = adjs.flatten(-2)

In [27]:
adjs_dgl = adjs_dgl.to("cuda:7")

In [30]:
%%timeit -n5 -r5
adjs_dgl_res_0 = dgl_knn_graph(adjs_dgl[:, 0], k=5, exclude_self=True)

2.26 ms ± 162 µs per loop (mean ± std. dev. of 5 runs, 5 loops each)


In [31]:
%%timeit -n5 -r5
adjs_dgl_res_1 = dgl_knn_graph(adjs_dgl[:, 1], k=5, exclude_self=True)

2.73 ms ± 344 µs per loop (mean ± std. dev. of 5 runs, 5 loops each)


In [46]:
adjs_dgl_res_0.edges()

(tensor([ 80,  46, 292,  ..., 141, 215, 166], device='cuda:7'),
 tensor([  0,   0,   0,  ..., 371, 371, 371], device='cuda:7'))

## pyg_knn

In [40]:
%%time
pyg_knn(adjs[:, 0].flatten(-2), adjs[:, 0].flatten(-2), k=5, num_workers=2)

CPU times: user 3.36 s, sys: 326 ms, total: 3.69 s
Wall time: 2.86 s


tensor([[  0,   0,   0,  ..., 371, 371, 371],
        [  0,  80,  46,  ..., 265, 141, 215]])

In [43]:
%%time
pyg_knn_graph(adjs[:, 0].flatten(-2).to('cuda:7'), k=5)

CPU times: user 2.57 s, sys: 194 ms, total: 2.77 s
Wall time: 2.22 s


tensor([[ 80,  46, 292,  ..., 141, 215, 166],
        [  0,   0,   0,  ..., 371, 371, 371]], device='cuda:7')